In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import statsmodels.api as stm
import statsmodels.stats.weightstats as ws
import math
import seaborn as sns
import os

from sklearn.preprocessing import StandardScaler 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix


# Import Gaussian Naive Bayes classifier:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, balanced_accuracy_score

from sklearn.metrics import roc_curve, auc

from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import OLSInfluence
from scipy import stats
from sklearn.model_selection  import train_test_split
import PyQt5 #works
from PyQt5 import QtCore, QtGui, QtWidgets #works for pyqt5
%matplotlib qt


In [2]:
print (os.getcwd())
currDir = os.getcwd()
fullPath = currDir + "/CabaneASucrev0r3.csv"
type(currDir)

/home/hadoop/Git_Ia/CegepSteFoy_IA/04_Eda/TP02


str

In [3]:
donnee = pd.read_csv(fullPath)
stats=donnee.describe()
dimensions=donnee.shape
donnee.columns.values

array(['Date', 'Année', 'Jour Calendrier Saison', 'Temp max.(°C)',
       'Temp min.(°C)', 'Temp moy.(°C)', 'Diff Temp (°C)',
       'Précip. tot. (mm)', 'Précip. Tot. Hiver (mm)',
       'Nombre épisodes gel/dégel', 'Production moyenne par entaille (L)',
       'Débit sève (L/j)', 'Sucre sève (%)',
       'Alimentation osmoseur (L/j)', 'Osmoseur (heures opération/j)',
       'Pression osmoseur (bar)', 'Sucre sortie osmoseur (%)',
       'Température Bouilloire (0C)', 'Temps bouilloire (h)',
       'Sucre du sirop obtenu (%)', 'Quantité de sirop obtenue (L)',
       'Transmittance produit (%)', 'Pixel1_R', 'Pixel1_G', 'Pixel1_B',
       'Pixel2_R', 'Pixel2_G', 'Pixel2_B', 'Pixel3_R', 'Pixel3_G',
       'Pixel3_B', 'Pixel4_R', 'Pixel4_G', 'Pixel4_B', 'Pixel5_R',
       'Pixel5_G', 'Pixel5_B', 'Pixel6_R', 'Pixel6_G', 'Pixel6_B',
       'Pixel7_R', 'Pixel7_G', 'Pixel7_B', 'Pixel8_R', 'Pixel8_G',
       'Pixel8_B', 'Pixel9_R', 'Pixel9_G', 'Pixel9_B', 'Classe Sirop'],
      dtype=object)

In [4]:
## éliminer les null (fill)
donnee.fillna(method='bfill', inplace=True)


In [5]:
## copier seulement les colonnes pixel
dfPixel = donnee[donnee.filter(like='Pixel').columns].copy()

dfPixel['Classe Sirop'] = donnee['Classe Sirop']
dfPixel.columns.values

array(['Pixel1_R', 'Pixel1_G', 'Pixel1_B', 'Pixel2_R', 'Pixel2_G',
       'Pixel2_B', 'Pixel3_R', 'Pixel3_G', 'Pixel3_B', 'Pixel4_R',
       'Pixel4_G', 'Pixel4_B', 'Pixel5_R', 'Pixel5_G', 'Pixel5_B',
       'Pixel6_R', 'Pixel6_G', 'Pixel6_B', 'Pixel7_R', 'Pixel7_G',
       'Pixel7_B', 'Pixel8_R', 'Pixel8_G', 'Pixel8_B', 'Pixel9_R',
       'Pixel9_G', 'Pixel9_B', 'Classe Sirop'], dtype=object)

In [6]:
## convertir Classe Sirop en numérique - https://www.youtube.com/watch?v=wH_ezgftiy0&t=136s
dfPixel['CategClasseSirop'] = dfPixel['Classe Sirop'].astype('category').cat.codes

In [7]:
dfPixel['CategClasseSirop'].unique()

array([0, 1, 3, 4, 2], dtype=int8)

In [8]:
## supprimer la classe 0
dfPixel = dfPixel[(dfPixel['CategClasseSirop']>0)]

In [9]:
sns.histplot(dfPixel['Classe Sirop'])

<AxesSubplot: xlabel='Classe Sirop', ylabel='Count'>

In [10]:
dfPixel[dfPixel['CategClasseSirop']==4].head()

,Pixel1_R,Pixel1_G,Pixel1_B,Pixel2_R,Pixel2_G,Pixel2_B,Pixel3_R,Pixel3_G,Pixel3_B,Pixel4_R,...,Pixel7_G,Pixel7_B,Pixel8_R,Pixel8_G,Pixel8_B,Pixel9_R,Pixel9_G,Pixel9_B,Classe Sirop,CategClasseSirop
43,0.00,40.37,42.54,0.00,35.71,58.70,0.00,28.01,63.79,0.00,...,76.93,42.04,0.00,81.50,108.42,0.00,57.218731,86.109115,Très Foncé,4
78,24.37,82.80,94.98,5.33,54.21,62.36,15.01,14.77,38.13,33.23,...,21.84,106.02,46.91,9.80,66.44,0.00,48.756847,55.409751,Très Foncé,4
135,70.64,29.68,51.62,48.20,30.49,82.84,25.18,62.96,43.09,55.89,...,11.84,94.76,45.76,33.56,73.37,52.85,11.017551,93.506519,Très Foncé,4
217,0.00,46.01,113.77,0.00,66.36,108.71,0.00,43.95,88.86,0.00,...,86.93,56.93,0.00,52.72,104.91,0.00,81.710235,95.545879,Très Foncé,4
231,36.72,38.32,46.71,58.75,10.82,73.07,24.67,57.97,75.49,52.25,...,28.11,48.85,33.97,43.53,84.73,55.90,78.311758,82.872750,Très Foncé,4


In [11]:
nom_col_y = 'CategClasseSirop'
dfX = dfPixel.select_dtypes(exclude='object').copy()
dfX = dfX.drop(nom_col_y, axis=1)
dfY = dfPixel[nom_col_y]
#dfX.columns.values
#dfPixel.dtypes

In [47]:
#### KNN
#### https://www.activestate.com/resources/quick-reads/how-to-classify-data-in-python/

# Split dataset into random train and test subsets:
X_train, X_test, y_train, y_test = train_test_split(dfX, dfY, test_size=0.20) 

# Standardize features by removing mean and scaling to unit variance:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test) 

# Use the KNN classifier to fit data:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train) 

# Predict y data with classifier: 
y_predict = classifier.predict(X_test)

# Print results: 
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict, target_names=dfPixel['Classe Sirop'].unique())) 

[[ 76   0   7   0]
 [  4   1   0   0]
 [  6   0 213   1]
 [  0   0   2   9]]
              precision    recall  f1-score   support

       Ambré       0.88      0.92      0.90        83
       Foncé       1.00      0.20      0.33         5
  Très Foncé       0.96      0.97      0.96       220
        Doré       0.90      0.82      0.86        11

    accuracy                           0.94       319
   macro avg       0.94      0.73      0.76       319
weighted avg       0.94      0.94      0.93       319



In [35]:
#### Naive Bayes
### https://www.activestate.com/resources/quick-reads/how-to-classify-data-in-python/

# Organize data:
label_names = dfPixel['Classe Sirop'].unique()
labels = dfY
feature_names = dfX.columns.values
features = dfX

In [52]:
# Split dataset into random train and test subsets:
train, test, train_labels, test_labels = train_test_split(features, labels, test_size=0.33, random_state=42)

# Initialize classifier:
gnb = GaussianNB()

# Train the classifier:
model = gnb.fit(train, train_labels)
# Make predictions with the classifier:
predictive_labels = gnb.predict(test)
print(predictive_labels)

# Evaluate label (subsets) accuracy:
print(accuracy_score(test_labels, predictive_labels))
print(balanced_accuracy_score(test_labels, predictive_labels))

[3 3 1 4 1 3 4 3 3 3 1 3 3 1 3 3 3 3 2 3 3 3 3 3 3 3 3 3 1 1 1 1 1 3 4 3 4
 3 4 1 3 1 3 3 1 2 1 1 3 1 3 3 1 3 3 1 1 3 3 3 3 3 1 3 3 1 1 1 3 1 3 3 3 1
 3 3 1 1 1 3 1 3 3 3 4 3 1 3 1 1 3 3 3 4 3 3 1 4 3 3 2 1 3 3 1 3 3 3 2 3 3
 3 4 3 1 3 1 3 3 3 1 3 3 1 3 3 1 1 1 4 3 3 3 3 3 3 3 3 3 1 1 1 3 3 1 3 4 3
 3 3 1 3 3 1 3 3 3 3 4 1 1 1 1 3 3 1 3 3 3 1 3 3 3 3 3 3 4 3 3 3 3 3 3 1 3
 3 3 3 3 1 1 1 3 3 1 3 3 3 1 3 3 3 3 3 1 3 3 3 4 1 3 3 4 1 1 3 3 3 3 3 3 1
 1 3 1 3 3 4 3 1 1 3 3 3 1 1 3 3 4 1 3 3 1 1 1 3 1 3 1 3 1 4 3 3 3 3 3 3 3
 3 1 3 3 3 3 3 1 1 3 3 3 1 3 3 3 3 3 1 3 3 3 1 1 3 4 3 3 3 1 3 1 1 3 1 3 3
 1 2 3 3 3 3 3 1 1 1 3 3 3 3 3 1 1 3 1 1 3 3 3 1 4 1 3 1 1 1 1 3 3 1 3 1 3
 3 1 3 3 3 1 3 3 1 3 1 3 1 1 1 1 3 4 3 3 3 3 1 1 1 1 3 1 3 1 1 1 3 3 3 3 3
 3 3 3 3 1 1 3 3 3 1 3 1 1 3 3 3 1 3 3 1 1 3 4 3 1 3 4 1 3 1 3 3 4 3 1 3 3
 4 3 3 1 4 3 3 3 1 3 3 3 1 3 4 2 1 3 1 3 3 3 3 3 1 3 1 3 1 3 1 1 4 1 3 3 3
 3 4 3 3 3 3 3 4 3 1 3 4 1 3 3 3 3 1 3 3 3 3 1 3 3 3 3 1 3 3 1 1 1 3 3 3 3
 1 3 3 3 1 3 3 3 3 3 3 3 

In [49]:
print(accuracy_score.__module__)

sklearn.metrics._classification


In [75]:
## https://www.youtube.com/watch?v=8gLewErTU24
def plot_confusion_matrix( pY_true, pY_pred):
    #labels = unique_labels(y_test)
    labels = dfPixel['Classe Sirop'].unique()
    cols = [f'Predictted{label}' for label in labels]
    rows = [f'Actual{label}' for label in labels]
    table = pd.DataFrame(confusion_matrix(pY_true, pY_pred)
                         , columns=cols, index=rows)
    print(table)
    sns.heatmap(table, annot=True, fmt=".2f", cmap='viridis')
    #return table

In [78]:
plot_confusion_matrix(y_test, y_predict )

                  PredicttedAmbré  PredicttedFoncé  PredicttedTrès Foncé  \
ActualAmbré                    76                0                     7   
ActualFoncé                     4                1                     0   
ActualTrès Foncé                6                0                   213   
ActualDoré                      0                0                     2   

                  PredicttedDoré  
ActualAmbré                    0  
ActualFoncé                    0  
ActualTrès Foncé               1  
ActualDoré                     9  
